In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession
    .builder
    .appName("Joins and Data Partitions")
    .master("local[*]")
    .getOrCreate()
)

spark

25/02/27 15:47:31 WARN Utils: Your hostname, codespaces-7aea8b resolves to a loopback address: 127.0.0.1; using 10.0.2.247 instead (on interface eth0)
25/02/27 15:47:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/27 15:47:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/02/27 15:47:33 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
# Emp Data & Schema

emp_data = [
    ["001","101","John Doe","30","Male","50000","2015-01-01"],
    ["002","101","Jane Smith","25","Female","45000","2016-02-15"],
    ["003","102","Bob Brown","35","Male","55000","2014-05-01"],
    ["004","102","Alice Lee","28","Female","48000","2017-09-30"],
    ["005","103","Jack Chan","40","Male","60000","2013-04-01"],
    ["006","103","Jill Wong","32","Female","52000","2018-07-01"],
    ["007","101","James Johnson","42","Male","70000","2012-03-15"],
    ["008","102","Kate Kim","29","Female","51000","2019-10-01"],
    ["009","103","Tom Tan","33","Male","58000","2016-06-01"],
    ["010","104","Lisa Lee","27","Female","47000","2018-08-01"],
    ["011","104","David Park","38","Male","65000","2015-11-01"],
    ["012","105","Susan Chen","31","Female","54000","2017-02-15"],
    ["013","106","Brian Kim","45","Male","75000","2011-07-01"],
    ["014","107","Emily Lee","26","Female","46000","2019-01-01"],
    ["015","106","Michael Lee","37","Male","63000","2014-09-30"],
    ["016","107","Kelly Zhang","30","Female","49000","2018-04-01"],
    ["017","105","George Wang","34","Male","57000","2016-03-15"],
    ["018","104","Nancy Liu","29","","50000","2017-06-01"],
    ["019","103","Steven Chen","36","Male","62000","2015-08-01"],
    ["020","102","Grace Kim","32","Female","53000","2018-11-01"]
]

emp_schema = "employee_id string, department_id string, name string, age string, gender string, salary string, hire_date string"

dept_data = [
    ["101", "Sales", "NYC", "US", "1000000"],
    ["102", "Marketing", "LA", "US", "900000"],
    ["103", "Finance", "London", "UK", "1200000"],
    ["104", "Engineering", "Beijing", "China", "1500000"],
    ["105", "Human Resources", "Tokyo", "Japan", "800000"],
    ["106", "Research and Development", "Perth", "Australia", "1100000"],
    ["107", "Customer Service", "Sydney", "Australia", "950000"]
]

dept_schema = "department_id string, department_name string, city string, country string, budget string"

In [3]:
# Create emp & dept DataFrame

emp = spark.createDataFrame(data=emp_data, schema=emp_schema)
dept = spark.createDataFrame(data=dept_data, schema=dept_schema)

In [4]:
# Show emp dataframe (ACTION)

emp.show()
dept.show()

+-----------+-------------+-------------+---+------+------+----------+
|employee_id|department_id|         name|age|gender|salary| hire_date|
+-----------+-------------+-------------+---+------+------+----------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|
|        006|          103|    Jill Wong| 32|Female| 52000|2018-07-01|
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|
|        008|          102|     Kate Kim| 29|Female| 51000|2019-10-01|
|        009|          103|      Tom Tan| 33|  Male| 58000|2016-06-01|
|        010|          104|     Lisa Lee| 27|Female| 47000|2018-08-01|
|        011|          104|   David Park| 38|  Male| 65000|2015-11-01|
|     

In [5]:
# Print Schema

emp.printSchema()
dept.printSchema()

root
 |-- employee_id: string (nullable = true)
 |-- department_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- hire_date: string (nullable = true)

root
 |-- department_id: string (nullable = true)
 |-- department_name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- budget: string (nullable = true)



In [6]:
# Get number of partitions for emp
emp.rdd.getNumPartitions()

2

In [7]:
# Get number of partitions for dept
dept.rdd.getNumPartitions()

2

In [9]:
emp_partitioned = emp.repartition(4)
emp_partitioned.rdd.getNumPartitions()

4

In [12]:
from pyspark.sql.functions import spark_partition_id
emp_partitioned = emp.repartition(4,'department_id')
emp_1 = emp_partitioned.withColumn('partionnum',spark_partition_id())
emp_1.show()

+-----------+-------------+-------------+---+------+------+----------+----------+
|employee_id|department_id|         name|age|gender|salary| hire_date|partionnum|
+-----------+-------------+-------------+---+------+------+----------+----------+
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|         0|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|         0|
|        008|          102|     Kate Kim| 29|Female| 51000|2019-10-01|         0|
|        014|          107|    Emily Lee| 26|Female| 46000|2019-01-01|         0|
|        016|          107|  Kelly Zhang| 30|Female| 49000|2018-04-01|         0|
|        020|          102|    Grace Kim| 32|Female| 53000|2018-11-01|         0|
|        012|          105|   Susan Chen| 31|Female| 54000|2017-02-15|         1|
|        017|          105|  George Wang| 34|  Male| 57000|2016-03-15|         1|
|        010|          104|     Lisa Lee| 27|Female| 47000|2018-08-01|         2|
|        011|   

In [13]:
# INNER JOIN datasets
# select e.emp_name, d.department_name, d.department_id, e.salary 
# from emp e inner join dept d on emp.department_id = dept.department_id
df_joined = emp.join(dept,how='inner',on=emp.department_id==dept.department_id)
df_joined.show()

+-----------+-------------+-------------+---+------+------+----------+-------------+--------------------+-------+---------+-------+
|employee_id|department_id|         name|age|gender|salary| hire_date|department_id|     department_name|   city|  country| budget|
+-----------+-------------+-------------+---+------+------+----------+-------------+--------------------+-------+---------+-------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|          101|               Sales|    NYC|       US|1000000|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|          101|               Sales|    NYC|       US|1000000|
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|          101|               Sales|    NYC|       US|1000000|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|          102|           Marketing|     LA|       US| 900000|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|      

In [15]:
# LEFT OUTER JOIN datasets
# select e.emp_name, d.department_name, d.department_id, e.salary 
# from emp e left outer join dept d on emp.department_id = dept.department_id
#df_left_joined = emp.join(dept,how='left_outer',on=emp.department_id==dept.department_id)
#using alias for names
df_left_joined = emp.alias('e').join(dept.alias('d'),how='left_outer',on=emp.department_id==dept.department_id)
df_left_joined.select('e.employee_id','d.department_id','e.name','e.salary').show()

+-----------+-------------+-------------+------+
|employee_id|department_id|         name|salary|
+-----------+-------------+-------------+------+
|        001|          101|     John Doe| 50000|
|        002|          101|   Jane Smith| 45000|
|        007|          101|James Johnson| 70000|
|        010|          104|     Lisa Lee| 47000|
|        003|          102|    Bob Brown| 55000|
|        004|          102|    Alice Lee| 48000|
|        008|          102|     Kate Kim| 51000|
|        005|          103|    Jack Chan| 60000|
|        006|          103|    Jill Wong| 52000|
|        009|          103|      Tom Tan| 58000|
|        014|          107|    Emily Lee| 46000|
|        016|          107|  Kelly Zhang| 49000|
|        011|          104|   David Park| 65000|
|        018|          104|    Nancy Liu| 50000|
|        020|          102|    Grace Kim| 53000|
|        019|          103|  Steven Chen| 62000|
|        013|          106|    Brian Kim| 75000|
|        015|       

In [18]:
# Bonus TIP
# Joins with cascading conditions
# Join with Department_id and only for departments 101 or 102
# Join with not null/null conditions
df_final = emp.alias('e').join(dept.alias('d'),how='left_outer',on=(emp.department_id==dept.department_id) & ((emp.department_id==101) |(emp.department_id==102))
                               &(emp.salary.isNotNull()) )
df_final.show()

+-----------+-------------+-------------+---+------+------+----------+-------------+---------------+----+-------+-------+
|employee_id|department_id|         name|age|gender|salary| hire_date|department_id|department_name|city|country| budget|
+-----------+-------------+-------------+---+------+------+----------+-------------+---------------+----+-------+-------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|          101|          Sales| NYC|     US|1000000|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|          101|          Sales| NYC|     US|1000000|
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|          101|          Sales| NYC|     US|1000000|
|        010|          104|     Lisa Lee| 27|Female| 47000|2018-08-01|         NULL|           NULL|NULL|   NULL|   NULL|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|          102|      Marketing|  LA|     US| 900000|
|        004|          1